# Librerías

In [1]:
import pandas as pd
import re

# Creación del contenido de la futura base de datos

In [2]:
df = pd.read_csv('./pelistop.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11508 entries, 0 to 11507
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   11508 non-null  int64  
 1   Título       11508 non-null  object 
 2   Año          11508 non-null  int64  
 3   Director/es  11508 non-null  object 
 4   Reparto      11508 non-null  object 
 5   Género/s     11508 non-null  object 
 6   País         11508 non-null  object 
 7   Puntuación   11508 non-null  float64
 8   Nº votos     11508 non-null  int64  
 9   Tipo         11508 non-null  object 
dtypes: float64(1), int64(3), object(6)
memory usage: 899.2+ KB


In [4]:
df.head(3)

,Unnamed: 0,Título,Año,Director/es,Reparto,Género/s,País,Puntuación,Nº votos,Tipo
0,0,Fahrenheit 9/11,2004,['Michael Moore'],"['Documental', ' (intervenciones de: Michael M...","['Documental', '11-S', 'Guerra de Iraq']",Estados Unidos,6.99,69775,Película
1,1,La noche más oscura (Zero Dark Thirty),2012,['Kathryn Bigelow'],"['Jessica Chastain', 'Joel Edgerton', 'Taylor ...","['Thriller', 'Acción', 'Drama', 'Terrorismo', ...",Estados Unidos,6.79,36704,Película
2,2,La última noche,2002,['Spike Lee'],"['Edward Norton', 'Philip Seymour Hoffman', 'B...","['Drama', 'Película de culto', '11-S']",Estados Unidos,7.49,33793,Película


In [7]:
df.drop(columns='Unnamed: 0', inplace=True)

In [8]:
type(df.loc[0,'Reparto'])

str

In [9]:
type(eval(df.loc[0,'Reparto']))

list

In [10]:
df.loc[0,'Reparto']

"['Documental', ' (intervenciones de: Michael Moore', 'George W. Bush', 'George Bush)']"

In [11]:
df['Reparto'] = df['Reparto'].str.replace("(\['Documental')[^\]]*(\])","['Documental'"+']')

In [12]:
df.loc[0,'Reparto']

"['Documental']"

#### Convertimos las columnas "Director/es", "Género/s" y "Reparto" a lista

In [13]:
df['Director/es'] = df['Director/es'].apply(lambda x: eval(x))

In [14]:
df['Género/s'] = df['Género/s'].apply(lambda x: eval(x))

In [15]:
df['Reparto'] = df['Reparto'].apply(lambda x: eval(x))

In [21]:
type(df.loc[0,'Reparto'])

list

#### Reseteamos el índice para poder asignar ese índice al id de la película

In [23]:
df.reset_index(inplace=True)

In [24]:
df.rename(columns={
    'index':'id_peli',
    'Título':'titulo',
    'Año':'año',
    'Director/es':'direccion',
    'Reparto':'actor',
    'Género/s':'genero',
    'País':'pais',
    'Puntuación':'puntuacion',
    'Nº votos':'votos',
    'Tipo': 'tipo'
}, inplace=True)

In [27]:
df.columns

Index(['id_peli', 'titulo', 'año', 'direccion', 'actor', 'genero', 'pais',
       'puntuacion', 'votos', 'tipo'],
      dtype='object')

## DataFrame: Películas

In [29]:
df_pelis = df[['id_peli', 'titulo', 'año', 'pais', 'tipo', 'puntuacion', 'votos']]

In [30]:
df_pelis.head()

,id_peli,titulo,año,pais,tipo,puntuacion,votos
0,0,Fahrenheit 9/11,2004,Estados Unidos,Película,6.99,69775
1,1,La noche más oscura (Zero Dark Thirty),2012,Estados Unidos,Película,6.79,36704
2,2,La última noche,2002,Estados Unidos,Película,7.49,33793
3,3,Zeitgeist: The Movie,2007,Estados Unidos,Película,7.36,21808
4,4,United 93 (Vuelo 93),2006,Estados Unidos,Película,6.70,19146


## DataFrame: Dirección

In [31]:
df_dir = df.explode('direccion')[['direccion']].drop_duplicates()

In [32]:
df_dir.reset_index(inplace=True)

In [33]:
df_dir.drop(columns='index', inplace=True)

In [34]:
df_dir.reset_index(inplace=True)

In [35]:
df_dir.rename(columns={'index':'id_dir'}, inplace=True)

In [36]:
df_dir.head()

,id_dir,direccion
0,0,Michael Moore
1,1,Kathryn Bigelow
2,2,Spike Lee
3,3,Peter Joseph
4,4,Paul Greengrass


## DataFrame: Actores

In [37]:
df_act = df.explode('actor')[['actor']].drop_duplicates()

In [38]:
df_act.reset_index(inplace=True)

In [39]:
df_act.drop(columns='index', inplace=True)

In [40]:
df_act.reset_index(inplace=True)

In [41]:
df_act.rename(columns={'index':'id_act'}, inplace=True)

In [42]:
df_act.head()

,id_act,actor
0,0,Documental
1,1,Jessica Chastain
2,2,Joel Edgerton
3,3,Taylor Kinney
4,4,Kyle Chandler


## DataFrame: Género

In [43]:
df_gen = df.explode('genero')[['genero']].drop_duplicates()

In [44]:
df_gen.reset_index(inplace=True)

In [45]:
df_gen.drop(columns='index', inplace=True)

In [46]:
df_gen.reset_index(inplace=True)

In [47]:
df_gen.rename(columns={'index':'id_gen'}, inplace=True)

In [48]:
df_gen.head()

,id_gen,genero
0,0,Documental
1,1,11-S
2,2,Guerra de Iraq
3,3,Thriller
4,4,Acción


## DataFrame: Películas - Dirección

In [49]:
peli_dir = pd.merge(df.explode('direccion'), df_dir, on='direccion', how='inner')[['id_peli','id_dir']]

In [50]:
peli_dir.head()

,id_peli,id_dir
0,0,0
1,2075,0
2,7476,0
3,1,1
4,2523,1


## DataFrame: Películas - Actores

In [51]:
peli_act = pd.merge(df.explode('actor'), df_act, on='actor', how='inner')[['id_peli','id_act']]

In [52]:
peli_act.head()

,id_peli,id_act
0,0,0
1,3,0
2,10,0
3,13,0
4,14,0


## DataFrame: Películas - Género

In [53]:
peli_gen = pd.merge(df.explode('genero'), df_gen, on='genero', how='inner')[['id_peli','id_gen']]

In [54]:
peli_gen.head()

,id_peli,id_gen
0,0,0
1,3,0
2,10,0
3,13,0
4,14,0


## Export a csv

In [55]:
df_pelis.to_csv('1-pelis.csv', index=False)

In [56]:
df_dir.to_csv('2-direccion.csv', index=False)

In [57]:
df_act.to_csv('3-actores.csv', index=False)

In [58]:
df_gen.to_csv('4-genero.csv', index=False)

In [61]:
peli_dir.to_csv('5-pelis-direccion.csv', index=False)

In [60]:
peli_act.to_csv('6-pelis-actores.csv', index=False)

In [62]:
peli_gen.to_csv('7-pelis-genero.csv', index=False)

# SQL

In [63]:
%load_ext sql

In [64]:
%sql sqlite:///movies.db

'Connected: @movies.db'

In [65]:
db_url = 'sqlite:///movies.db'

In [74]:
%%sql
SELECT tbl_name
FROM sqlite_master
WHERE type='table'

 * sqlite:///movies.db
Done.


tbl_name
Pelis
Direccion
Genero
Actores
Pelis-Direccion
Pelis-Genero
Pelis-Actores


In [66]:
df_pelis.to_sql('Pelis', db_url, if_exists='append', index=False)

In [67]:
df_dir.to_sql('Direccion', db_url, if_exists='append', index=False)

In [69]:
df_gen.to_sql('Genero', db_url, if_exists='append', index=False)

In [68]:
df_act.to_sql('Actores', db_url, if_exists='append', index=False)

In [70]:
peli_dir.to_sql('Pelis-Direccion', db_url, if_exists='append', index=False)

In [72]:
peli_gen.to_sql('Pelis-Genero', db_url, if_exists='append', index=False)

In [71]:
peli_act.to_sql('Pelis-Actores', db_url, if_exists='append', index=False)

# 1. País con más producción de películas
**Visualización**: mapa. Cuando pasamos el cursor por encima, muestra las películas más valoradas de ese país.

Bases de datos necesarias: _Pelis_

# 2. Géneros más populares por año
**Visualización**: cuando pasamos el cursor por encima, muestra las películas más valoradas de ese género.

Bases de datos necesarias: _Pelis, Genero, Pelis-Genero_

# 3. Top 3 actores y directores mejor valorados
**Visualización**: Por periodo de tiempo, top 3 actores y directores cuya puntuación media de las peliculas en las que han participado sea más alta

Bases de datos necesarias: _Pelis, Direccion, Actores, Pelis-Direccion, Pelis-Actores_